In [1]:
#!sudo apt-get install -y xvfb ffmpeg
!pip install atari_py
!pip install gym[atari]
!pip install imageio
!pip install PILLOW
!pip install pyglet
!pip install typing-extensions==3.7.4.3
!pip install pyvirtualdisplay
!pip install tf-agents
!pip install ../../OSAR-keras/.
# !pip install git+https://github.com/ustyuzhaninky/OSAR-keras

Processing /mnt/d/jorgen/projects/python/PhDSub/TowerProject/OSAR-keras


  Created wheel for OSAR: filename=OSAR-0.1.7-py3-none-any.whl size=65574 sha256=e7a499c922d2d7cf029daab567707c6374bc1b9b7fdeb2df2de0a067d54716e4
  Stored in directory: /home/jorgen/.cache/pip/wheels/dc/71/09/748828c7ebd4ea8d8e8ce637e1142f8d6a97aab86f28cd6eb1
Successfully built OSAR
  Attempting uninstall: OSAR
    Found existing installation: OSAR 0.1.7
    Uninstalling OSAR-0.1.7:
      Successfully uninstalled OSAR-0.1.7


In [2]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay

import tensorflow as tf

from tf_agents import agents
from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import q_network
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

from OSAR import OSARNetwork, Runner

In [3]:
env_names = [
    'AirRaid-ram-v0',
    'Solaris-ram-v0',
    'Alien-ram-v0',
    'Berzerk-ram-v0',
    'Asteroids-ram-v0',
    'Venture-ram-v0',
    'MontezumaRevenge-ram-v0',
    'Asteroids-ram-v0',
    'BattleZone-ram-v0',
    'Enduro-ram-v0',
    'Tutankham-ram-v0',
    'WizardOfWor-ram-v0'
]

In [4]:
num_iterations = 10000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"} 
collect_steps_per_iteration = 1  # @param {type:"integer"}
replay_buffer_max_length = 1000  # @param {type:"integer"}

batch_size = 32  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 10  # @param {type:"integer"}
memory_len = 10 # @param {type: "integer"}
n_turns = 5 # @param {type: "integer"}
num_atoms = 21  # @param {type:"integer"}
min_q_value = -10  # @param {type:"integer"}
max_q_value = 10  # @param {type:"integer"}
n_step_update = 2  # @param {type:"integer"}

num_eval_episodes = 1000  # @param {type:"integer"}
eval_interval = 10  # @param {type:"integer"}


In [5]:
from tf_agents.networks import categorical_q_network
fc_layer_params = (10,)
from tf_agents.networks import categorical_q_network

agent_specs = {
    'batch_size': 1,
    'memory_len': 2,
    'n_turns': 3,
    'fc_layer_params': fc_layer_params,
    'num_atoms': 21,
    'conv_type': '1d',
    'learning_rate': 1e-3,
    'q_value': 10,
    'n_step_update': 2,
}

def osar_generator(
    observation_spec,
    action_spec,
    batch_size,
    memory_len,
    n_turns,
    fc_layer_params,
    num_atoms,
    conv_type,
    learning_rate,
    time_step_spec,
    n_step_update,
    q_value=10,
    ):
    q_net = OSARNetwork(
        observation_spec,
        action_spec,
        batch_size,
        memory_len,
        n_turns,
        fc_layer_params=fc_layer_params,
        num_atoms=num_atoms,
        conv_type='1d',
        )
#     q_net = categorical_q_network.CategoricalQNetwork(
#     observation_spec,
#     action_spec,
#     num_atoms=num_atoms,
#     fc_layer_params=fc_layer_params)
    
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

    train_step_counter = tf.Variable(0)

    agent = agents.categorical_dqn.categorical_dqn_agent.CategoricalDqnAgent(
        time_step_spec,
        action_spec,
        categorical_q_network=q_net,
        optimizer=optimizer,
        td_errors_loss_fn=common.element_wise_squared_loss,
        train_step_counter=train_step_counter,
        min_q_value = -min_q_value,
        max_q_value = min_q_value,
        n_step_update = n_step_update,
    )
    agent.initialize()
    
    return agent

    

In [6]:
experiment_common_specs = {
    'agent_specs': agent_specs,
    'agent_generator': osar_generator,
    'num_iterations': num_iterations,
    'initial_collect_steps': initial_collect_steps,
    'collect_steps_per_iteration': collect_steps_per_iteration,
    'replay_buffer_max_length': replay_buffer_max_length,
    'num_eval_episodes': num_eval_episodes,
    'eval_interval': eval_interval,
    'n_step_update': n_step_update,
}

In [7]:
configs = []
for name in env_names:
    config = experiment_common_specs.copy()
    config['env_name'] = name
    configs.append(config)

In [8]:
%%time
runner = Runner(model_name='test_one', logpath='', list_configs=configs)
runner.run(progress=False, experiment_progress=True)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


  0%|          | 0/4 [00:00<?, ?it/s]

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


Episode 3: 100%|██████████| 4/4 [02:07<00:00, 31.94s/it, avg_return=209, train_loss=158]


Execution Finished.
CPU times: user 3min 18s, sys: 24.1 s, total: 3min 42s
Wall time: 3min 18s
